In [19]:
!pip install googletrans==4.0.0-rc1


In [20]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from googletrans import Translator

In [21]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
from google.colab import files
uploaded = files.upload()

Saving coffee_analysis.csv to coffee_analysis (1).csv


In [23]:
import io
df = pd.read_csv(io.BytesIO(next(iter(uploaded.values()))))

In [24]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [25]:
def preprocess_text(text):
    # Check if the input is a string before processing
    if not isinstance(text, str):
        return "" # Return an empty string or handle non-string data as appropriate

    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in string.punctuation]
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words]
    return ' '.join(tokens)

# Ensure 'desc_1' column exists before applying the function
if 'desc_1' in df.columns:
    df['texto_tratado'] = df['desc_1'].apply(preprocess_text)
else:
    print("Error: 'desc_1' column not found in the DataFrame.")

In [26]:
vetorizador = TfidfVectorizer()
matriz_tfidf = vetorizador.fit_transform(df['texto_tratado'])
cosine_sim = cosine_similarity(matriz_tfidf, matriz_tfidf)

In [27]:
def traduzir_texto(texto):
    tradutor = Translator()
    try:
        traducao = tradutor.translate(texto, src='en', dest='pt')
        return traducao.text
    except:
        return texto

In [28]:
def recomendar_cafes(indice_cafe, top_n=5):
    similares = list(enumerate(cosine_sim[indice_cafe]))
    similares = sorted(similares, key=lambda x: x[1], reverse=True)[1:top_n+1]

    print("="*100)
    print(f"Café de referência:\n")
    print(f"[{indice_cafe}] Nome: {df.iloc[indice_cafe]['name']}")
    print("Avaliação (em português):")
    print(traduzir_texto(df.iloc[indice_cafe]['desc_1']))
    print("="*100)
    print("\nRecomendações similares:\n")

    for i, score in similares:
        nome = df.iloc[i]['name']
        desc = df.iloc[i]['desc_1']
        traducao = traduzir_texto(desc)
        print(f"[{i}] Nome: {nome}")
        print(f"Similaridade: {score:.2f}")
        print(f"Avaliação (em português): {traducao}")
        print("—" * 100)


In [29]:
def buscar_por_nome(nome_busca):
    resultados = df[df['name'].str.lower().str.contains(nome_busca.lower())]
    if resultados.empty:
        print("Nenhum café encontrado com esse nome.")
        return
    print("Cafés encontrados:")
    for idx, row in resultados.iterrows():
        print(f"[{idx}] {row['name']}")

    try:
        indice_escolhido = int(input("Digite o número (índice) do café para ver recomendações: "))
        if indice_escolhido in df.index:
            recomendar_cafes(indice_cafe=indice_escolhido, top_n=3)
        else:
            print("Índice inválido.")
    except ValueError:
        print("Entrada inválida. Digite apenas um número.")


nome_digitado = input("Digite o nome (ou parte do nome) do café que deseja buscar: ")
buscar_por_nome(nome_digitado)


Digite o nome (ou parte do nome) do café que deseja buscar: catuai
Cafés encontrados:
[595] Panama Elida Natural Catuai
[1123] Honduras Catuai/Bourbon
[1426] Guatemala Finca El Principito Yellow Catuai
[1516] Elida Estate Dragonfly Catuai Lot 13
[1519] Elida Estate ASD Natural Catuai 15
Digite o número (índice) do café para ver recomendações: 1123
Café de referência:

[1123] Nome: Honduras Catuai/Bourbon
Avaliação (em português):
Crisply Sweet, em tons de nozes.Nutella, ameixa, cedro, raspas de laranja, um toque de lírio em aroma e copo.Estrutura doce com acidez suave e redonda;Fela da boca aveludada e suave.Acabamento em tons de floral e noz.

Recomendações similares:

[2058] Nome: Esperanza Villa Sarchi
Similaridade: 0.69
Avaliação (em português): Crisply Sweet, em tons de nozes.Nutella, pêssego, raspas de limão, cedro, magnólia em aroma e copo.Estrutura doce com acidez suave e redonda;Fela da boca aveludada e suave.Acabamento suavemente seco e com tons de nozes muito doces.
————————